In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas
from unidecode import unidecode
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def ini_data():
    data = {
        'Id Magasin':[],
        'Enseigne':[],
        'Code Produit':[],
        'Libelle':[],
        'Prix':[],
        'Prix Unitaire':[],
        'Unite Px Unitaire':[],
        'Prix Num':[],
        'DateRecup':[],
        'HeureRecup':[],
        'Lien Photo':[],
        'Lien':[],
    }
    return data

In [ ]:
ref_ids = """92010366
92011384
92012423
92025294
92025294
92020900
92002347
92009897
92009897
92013184
25023108
92025060
92025062"""
ref_ids = ref_ids.split("\n")

In [ ]:
def get_subitems(driver):
    sub_items = driver.find_elements_by_xpath("//tr[@class='ligne-multiproduit']")
    return sub_items
    
def get_libelle(driver):
    lib = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='product-name']")))
    #lib = driver.find_element_by_xpath("//div[@class='product-name']")
    return lib.text.strip()

def get_sublibelle(item, items_cuant):
    if not items_cuant:
        return ""
    lib1 = item.find_element_by_xpath("td")
    return lib1.text.strip()

def get_main_prix(item):
    price = item.find_element_by_xpath(".//span[@class='price']")
    if not price:
        return "-"
    else:
        return unidecode(price.text.strip())

def get_ref(item, items_cuant, ref):
    if not items_cuant:
        return ref
    ref2 = item.find_element_by_xpath("td[@class='sku']")
    return ref2.text.strip()
    
def get_prix_per_unit(item):
    price2 = item.find_elements_by_xpath(".//span[@class='price-total ']")
    if price2 and "soit" in price2[0].text and "/" in price2[0].text:
        return unidecode(price2[0].text.split("soit")[1])
    return "-"

def get_prix_unit(item):
    price2 = item.find_elements_by_xpath(".//span[@class='price-total ']")
    if price2 and "soit" in price2[0].text and "/" in price2[0].text:
        return price2[0].text.split("/")[1]
    return "-"

def get_photo(driver):
    photo = driver.find_element_by_xpath("//div[@class='zoomPad']/img")
    return photo.get_attribute("src")


In [ ]:
def scrap_products(driver, data, ref):
    while True:
        check = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='block-search']"))) 
        if ref in check.text:
            break
    action = webdriver.ActionChains(driver)
    prod = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='item first']")))
    action.move_by_offset(1, 1).perform()
    action.move_to_element(prod).perform()
    prod.click()
    obtain_info(driver, data, ref)

In [ ]:
def obtain_info(driver, data, ref):
    while True:
        try:
            libelle = get_libelle(driver)
            photo = get_photo(driver)
            items = get_subitems(driver)
            break
        except Exception as e:
            pass
    if not items:
        items = [driver]
    for item in items:
        data["Id Magasin"].append(shop_id)
        data["Enseigne"].append(shop_city)
        data["Code Produit"].append(get_ref(item, len(items) - 1, ref))
        data["Libelle"].append(" ".join([libelle] + [get_sublibelle(item, len(items) - 1)]))
        data["Prix"].append(get_main_prix(item))
        data["Prix Unitaire"].append(get_prix_per_unit(item))
        data["Unite Px Unitaire"].append(get_prix_unit(item))
        data["Prix Num"].append(".".join(data["Prix"][-1].split("€")))
        data["DateRecup"].append(datetime.datetime.today().strftime("%Y/%m/%d"))
        data["HeureRecup"].append(datetime.datetime.today().strftime("%H:%M:%S.%s"))
        data["Lien Photo"].append(photo)
        data["Lien"].append(driver.current_url)

In [ ]:
shop_id = "miramas"
shop_city = "Miramas"
data = ini_data()
driver = webdriver.Firefox()
driver.get(f"https://www.bricocash.fr/{shop_id}?___store={shop_id}")
for ref in ref_ids:
    searchbar = driver.find_element_by_id('search')
    searchbar.send_keys(Keys.CONTROL + 'a')
    searchbar.send_keys(Keys.BACKSPACE)
    searchbar.send_keys(ref)
    searchbar.send_keys(Keys.ENTER)
    scrap_products(driver, data, ref)
driver.close()
driver.quit()
pandas.DataFrame(data).to_csv(f"output_bricocash.csv", sep=';', index=False)


In [ ]:
df = pandas.DataFrame(data)

In [ ]:
df